# Obtención WEB de los datos del reporte via API - ENDPOINT

In [1]:
import requests
import pandas as pd
import time
import random
import os
from requests.exceptions import ConnectionError, Timeout

In [4]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20250101",
    "fechaFinal": "20250131",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

🚀 Iniciando descarga masiva...
✅ Pág 1 OK (150 reg). Total: 150
✅ Pág 2 OK (150 reg). Total: 300
✅ Pág 3 OK (150 reg). Total: 450
✅ Pág 4 OK (150 reg). Total: 600
✅ Pág 5 OK (150 reg). Total: 750
✅ Pág 6 OK (150 reg). Total: 900
✅ Pág 7 OK (150 reg). Total: 1050
✅ Pág 8 OK (150 reg). Total: 1200
✅ Pág 9 OK (150 reg). Total: 1350
✅ Pág 10 OK (150 reg). Total: 1500
✅ Pág 11 OK (150 reg). Total: 1650
✅ Pág 12 OK (150 reg). Total: 1800
✅ Pág 13 OK (150 reg). Total: 1950
✅ Pág 14 OK (150 reg). Total: 2100
✅ Pág 15 OK (150 reg). Total: 2250
✅ Pág 16 OK (150 reg). Total: 2400
✅ Pág 17 OK (150 reg). Total: 2550
✅ Pág 18 OK (150 reg). Total: 2700
✅ Pág 19 OK (150 reg). Total: 2850
✅ Pág 20 OK (150 reg). Total: 3000
✅ Pág 21 OK (150 reg). Total: 3150
✅ Pág 22 OK (150 reg). Total: 3300
✅ Pág 23 OK (150 reg). Total: 3450
✅ Pág 24 OK (150 reg). Total: 3600
✅ Pág 25 OK (150 reg). Total: 3750
✅ Pág 26 OK (150 reg). Total: 3900
✅ Pág 27 OK (150 reg). Total: 4050
✅ Pág 28 OK (150 reg). Total: 4200
✅ Pá

In [5]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20250201",
    "fechaFinal": "20250228",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_2 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

🚀 Iniciando descarga masiva...
✅ Pág 1 OK (150 reg). Total: 150
✅ Pág 2 OK (150 reg). Total: 300
✅ Pág 3 OK (150 reg). Total: 450
✅ Pág 4 OK (150 reg). Total: 600
✅ Pág 5 OK (150 reg). Total: 750
✅ Pág 6 OK (150 reg). Total: 900
✅ Pág 7 OK (150 reg). Total: 1050
✅ Pág 8 OK (150 reg). Total: 1200
✅ Pág 9 OK (150 reg). Total: 1350
❗ Error de conexión en pág 10. Reintento 1/5 en 20s...
✅ Pág 10 OK (150 reg). Total: 1500
✅ Pág 11 OK (150 reg). Total: 1650
✅ Pág 12 OK (150 reg). Total: 1800
✅ Pág 13 OK (150 reg). Total: 1950
✅ Pág 14 OK (150 reg). Total: 2100
✅ Pág 15 OK (150 reg). Total: 2250
✅ Pág 16 OK (150 reg). Total: 2400
✅ Pág 17 OK (150 reg). Total: 2550
✅ Pág 18 OK (150 reg). Total: 2700
✅ Pág 19 OK (150 reg). Total: 2850
✅ Pág 20 OK (150 reg). Total: 3000
✅ Pág 21 OK (150 reg). Total: 3150
✅ Pág 22 OK (150 reg). Total: 3300
✅ Pág 23 OK (150 reg). Total: 3450
✅ Pág 24 OK (150 reg). Total: 3600
✅ Pág 25 OK (150 reg). Total: 3750
✅ Pág 26 OK (150 reg). Total: 3900
✅ Pág 27 OK (150 re

In [6]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20250301",
    "fechaFinal": "20250331",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_3 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

🚀 Iniciando descarga masiva...
✅ Pág 1 OK (150 reg). Total: 150
✅ Pág 2 OK (150 reg). Total: 300
✅ Pág 3 OK (150 reg). Total: 450
✅ Pág 4 OK (150 reg). Total: 600
✅ Pág 5 OK (150 reg). Total: 750
✅ Pág 6 OK (150 reg). Total: 900
✅ Pág 7 OK (150 reg). Total: 1050
✅ Pág 8 OK (150 reg). Total: 1200
✅ Pág 9 OK (150 reg). Total: 1350
✅ Pág 10 OK (150 reg). Total: 1500
✅ Pág 11 OK (150 reg). Total: 1650
✅ Pág 12 OK (150 reg). Total: 1800
✅ Pág 13 OK (150 reg). Total: 1950
✅ Pág 14 OK (150 reg). Total: 2100
✅ Pág 15 OK (150 reg). Total: 2250
✅ Pág 16 OK (150 reg). Total: 2400
✅ Pág 17 OK (150 reg). Total: 2550
✅ Pág 18 OK (150 reg). Total: 2700
✅ Pág 19 OK (150 reg). Total: 2850
✅ Pág 20 OK (150 reg). Total: 3000
✅ Pág 21 OK (150 reg). Total: 3150
✅ Pág 22 OK (150 reg). Total: 3300
✅ Pág 23 OK (150 reg). Total: 3450
✅ Pág 24 OK (150 reg). Total: 3600
✅ Pág 25 OK (150 reg). Total: 3750
✅ Pág 26 OK (150 reg). Total: 3900
✅ Pág 27 OK (150 reg). Total: 4050
✅ Pág 28 OK (150 reg). Total: 4200
✅ Pá

In [7]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20250401",
    "fechaFinal": "20250430",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_4 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

🚀 Iniciando descarga masiva...
✅ Pág 1 OK (150 reg). Total: 150
❗ Error de conexión en pág 2. Reintento 1/5 en 20s...
✅ Pág 2 OK (150 reg). Total: 300
✅ Pág 3 OK (150 reg). Total: 450
✅ Pág 4 OK (150 reg). Total: 600
✅ Pág 5 OK (150 reg). Total: 750
✅ Pág 6 OK (150 reg). Total: 900
✅ Pág 7 OK (150 reg). Total: 1050
✅ Pág 8 OK (150 reg). Total: 1200
✅ Pág 9 OK (150 reg). Total: 1350
✅ Pág 10 OK (150 reg). Total: 1500
✅ Pág 11 OK (150 reg). Total: 1650
✅ Pág 12 OK (150 reg). Total: 1800
✅ Pág 13 OK (150 reg). Total: 1950
✅ Pág 14 OK (150 reg). Total: 2100
✅ Pág 15 OK (150 reg). Total: 2250
✅ Pág 16 OK (150 reg). Total: 2400
✅ Pág 17 OK (150 reg). Total: 2550
✅ Pág 18 OK (150 reg). Total: 2700
❗ Error de conexión en pág 19. Reintento 1/5 en 20s...
✅ Pág 19 OK (150 reg). Total: 2850
✅ Pág 20 OK (150 reg). Total: 3000
✅ Pág 21 OK (150 reg). Total: 3150
✅ Pág 22 OK (150 reg). Total: 3300
✅ Pág 23 OK (150 reg). Total: 3450
✅ Pág 24 OK (150 reg). Total: 3600
✅ Pág 25 OK (150 reg). Total: 3750


In [8]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20250501",
    "fechaFinal": "20250531",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_5 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

🚀 Iniciando descarga masiva...
✅ Pág 1 OK (150 reg). Total: 150
✅ Pág 2 OK (150 reg). Total: 300
✅ Pág 3 OK (150 reg). Total: 450
✅ Pág 4 OK (150 reg). Total: 600
✅ Pág 5 OK (150 reg). Total: 750
✅ Pág 6 OK (150 reg). Total: 900
✅ Pág 7 OK (150 reg). Total: 1050
✅ Pág 8 OK (150 reg). Total: 1200
✅ Pág 9 OK (150 reg). Total: 1350
✅ Pág 10 OK (150 reg). Total: 1500
✅ Pág 11 OK (150 reg). Total: 1650
✅ Pág 12 OK (150 reg). Total: 1800
✅ Pág 13 OK (150 reg). Total: 1950
✅ Pág 14 OK (150 reg). Total: 2100
✅ Pág 15 OK (150 reg). Total: 2250
✅ Pág 16 OK (150 reg). Total: 2400
✅ Pág 17 OK (150 reg). Total: 2550
✅ Pág 18 OK (150 reg). Total: 2700
✅ Pág 19 OK (150 reg). Total: 2850
✅ Pág 20 OK (150 reg). Total: 3000
✅ Pág 21 OK (150 reg). Total: 3150
✅ Pág 22 OK (150 reg). Total: 3300
✅ Pág 23 OK (150 reg). Total: 3450
✅ Pág 24 OK (150 reg). Total: 3600
✅ Pág 25 OK (150 reg). Total: 3750
✅ Pág 26 OK (150 reg). Total: 3900
✅ Pág 27 OK (150 reg). Total: 4050
✅ Pág 28 OK (150 reg). Total: 4200
✅ Pá

In [9]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20250601",
    "fechaFinal": "20250630",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_6 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

🚀 Iniciando descarga masiva...
✅ Pág 1 OK (150 reg). Total: 150
✅ Pág 2 OK (150 reg). Total: 300
✅ Pág 3 OK (150 reg). Total: 450
✅ Pág 4 OK (150 reg). Total: 600
✅ Pág 5 OK (150 reg). Total: 750
✅ Pág 6 OK (150 reg). Total: 900
✅ Pág 7 OK (150 reg). Total: 1050
✅ Pág 8 OK (150 reg). Total: 1200
✅ Pág 9 OK (150 reg). Total: 1350
✅ Pág 10 OK (150 reg). Total: 1500
✅ Pág 11 OK (150 reg). Total: 1650
✅ Pág 12 OK (150 reg). Total: 1800
✅ Pág 13 OK (150 reg). Total: 1950
✅ Pág 14 OK (150 reg). Total: 2100
✅ Pág 15 OK (150 reg). Total: 2250
✅ Pág 16 OK (150 reg). Total: 2400
✅ Pág 17 OK (150 reg). Total: 2550
✅ Pág 18 OK (150 reg). Total: 2700
✅ Pág 19 OK (150 reg). Total: 2850
✅ Pág 20 OK (150 reg). Total: 3000
✅ Pág 21 OK (150 reg). Total: 3150
✅ Pág 22 OK (150 reg). Total: 3300
✅ Pág 23 OK (150 reg). Total: 3450
✅ Pág 24 OK (150 reg). Total: 3600
✅ Pág 25 OK (150 reg). Total: 3750
✅ Pág 26 OK (150 reg). Total: 3900
✅ Pág 27 OK (150 reg). Total: 4050
❗ Error de conexión en pág 28. Reintent

In [ ]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20250701",
    "fechaFinal": "20250731",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_7 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

In [ ]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20250801",
    "fechaFinal": "20250831",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_8 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

In [ ]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20250901",
    "fechaFinal": "20250930",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_9 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

In [ ]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20251001",
    "fechaFinal": "20251031",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_10 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

In [ ]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20251101",
    "fechaFinal": "20251130",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_11 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")

In [ ]:
# --- CONFIGURACIÓN ---
URL = "https://portal.directoapp.pe/integracion_admin_prod/api/reporteAuthorization/detallado"
# 1. PEGA TU COOKIE AQUÍ
MI_COOKIE = "_ga=GA1.2.1157298828.1767931627; _gid=GA1.2.667079853.1767931627; auth_token=null; io=n-XGY76PGknazd-qD4PL" 

HEADERS = {
    "Content-Type": "application/json; charset=utf-8",
    "Cookie": MI_COOKIE,
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36",
    "Referer": "https://portal.directoapp.pe/admin-dev/default/reporte-empresarial"
}

# 2. PAYLOAD (Sacado de tu captura)
payload = {
    "idEmpresa": -1,
    "idCentroCosto": -1,
    "idCliente": -1,
    "fechaInicial": "20251201",
    "fechaFinal": "20251231",
    "campoDinamico": -1,
    "compania": -1,
    "idConductor": "",
    "idEstadoServicio": 7,
    "idServicio": -1,
    "idTipoPago": -1,
    "idTipoServicio": -1,
    "idUsuario": 534459,
    "isKmh": False,
    "page": 1,
    "peaje": -1,
    "rows": 150
}

datos_acumulados = []
pags_totales = 600 # Ajusta según cuánta info necesites

print("🚀 Iniciando descarga masiva...")

try:
    for i in range(1, pags_totales + 1):
        payload["page"] = i
        payload["draw"] = i
        exito_en_pagina = False
        reintentos = 0
        
        # Bucle de reintentos por página
        while not exito_en_pagina and reintentos < 5:
            try:
                # Timeout de 30s para evitar que el script se cuelgue infinito
                response = requests.post(URL, json=payload, headers=HEADERS, timeout=30)
                
                if response.status_code == 200:
                    res_json = response.json()
                    batch = res_json.get('data', res_json.get('rows', res_json.get('registros', [])))
                    
                    if not batch:
                        print(f"🏁 No se encontraron más datos en la página {i}.")
                        i = pags_totales + 1 # Para salir del bucle principal
                        break
                        
                    datos_acumulados.extend(batch)
                    ultima_pagina_exitosa = i
                    print(f"✅ Pág {i} OK ({len(batch)} reg). Total: {len(datos_acumulados)}")
                    exito_en_pagina = True
                    
                elif response.status_code in [500, 502, 503, 504]:
                    reintentos += 1
                    espera = reintentos * 15
                    print(f"⚠️ Error servidor {response.status_code} en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                    time.sleep(espera)
                
                elif response.status_code == 401:
                    print("❌ Cookie caducada. Deteniendo para actualizar.")
                    raise Exception("Cookie Expirada")

                else:
                    print(f"❌ Error inesperado {response.status_code} en pág {i}.")
                    break

            except (ConnectionError, Timeout) as e:
                reintentos += 1
                espera = reintentos * 20
                print(f"❗ Error de conexión en pág {i}. Reintento {reintentos}/5 en {espera}s...")
                time.sleep(espera)

        if not exito_en_pagina and reintentos >= 5:
            print(f"🛑 Se agotaron los reintentos para la página {i}. Guardando progreso...")
            break

        # Guardado de seguridad cada 100 páginas
        if i % 100 == 0:
            pd.DataFrame(datos_acumulados).to_excel("respaldo_seguridad.xlsx", index=False)
            with open("progreso_log.txt", "w") as f:
                f.write(f"Ultima pagina descargada: {i}")
            print(f"💾 Respaldo y LOG guardados en página {i}.")

        # Pausa aleatoria para evitar detección de comportamiento robótico
        time.sleep(random.uniform(1.2, 2.8))

except KeyboardInterrupt:
    print("\n⚠️ Proceso pausado por el usuario.")
except Exception as e:
    print(f"\n❌ Error fatal: {e}")

# --- CONSOLIDACIÓN FINAL ---
if datos_acumulados:
    df_12 = pd.DataFrame(datos_acumulados)
    print(f"\n✨ Extracción finalizada. {len(df)} registros listos para SQL Server.")
else:
    print("\n☹️ No se recolectaron datos.")